In [1]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps, loads

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = 'aacuser'
password = 'Enewcomb'
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso Salvare')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'grazioso-logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
app.layout = html.Div([
# FIX ME Also remember to include a unique identifier such as your name or date
# html.Div(id='hidden-div', style={'display':'none'}),

    html.Center(
        html.B(
            html.H1('Grazioso Salvare - Animal Shelter'))
    ),
    html.Center(
        html.B(
            html.H4('SNHU CS-340 - Elora')) # TODO: insert unique identifier code here
    ),
    html.Center(
        html.Img(id='logo', src='data:image/png;base64,{}'
        # Customer Logo
            .format(encoded_image.decode()), alt='Grazioso Salvare Logo', 
                style={'height': '10%', 'width': '10%'}
    )),       
    html.Hr(),
    
#Filter
    html.Div(
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        children=[
            #hmtl.Label('options'),
            dcc.Dropdown(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                    {'label': 'None', 'value': 'none'},
                ],
                multi=False,
                value='none',
            )]
    ),    
    html.Hr(),
    
# DataTable
    dt.DataTable(
        id='datatable-id',
        columns=[
            {'name': i, 'id': i, 'deletable': False, 'selectable': True}
                for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        style_as_list_view = True,
        fixed_rows={'headers': True},
        style_table={
            'height': '600px',
            'overflowY': 'auto', 
            'overflowX': 'auto'
        },
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable='single',
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_size=20
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={
            'display' : 'flex',
            'width': '100%'
        },
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style={ 'flex': 1 }
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={ 'flex': 1 }
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [
        Output('datatable-id', 'data'),
        Output('datatable-id', 'columns')
    ],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    print("HELP")
    if (filter_type == 'Water'):
        query = {'$and': [
            {'sex_upon_outcome': 'Intact Female'},
            {'age_upon_outcome_in_weeks': {'$gte': 26}},
            {'age_upon_outcome_in_weeks': {'$lte': 156}},
            {'$or': [
                {'breed': 'Labrador Retriever Mix'},
                {'breed': 'Chesapeake Bay Retriever'},
                {'breed': 'Newfoundland'},
            ]}
        ]}
        
    elif (filter_type == 'Outdoor'):
        query = {'$and': [
            {'sex_upon_outcome': 'Intact Male'},
            {'age_upon_outcome_in_weeks': {'$gte': 26}},
            {'age_upon_outcome_in_weeks': {'$lte': 156}},
            {'$or': [
                {'breed': 'German Shepherd'},
                {'breed': 'Alaskan Malamute'},
                {'breed': 'Old English Sheepdog'},
                {'breed': 'Siberian Husky'},
                {'breed': 'Rottweiler'}
            ]},
        ]}

    elif (filter_type == 'Disaster'):
        query = {'$and': [
            {'sex_upon_outcome': 'Intact Male'},
            {'age_upon_outcome_in_weeks': {'$gte': 20}},
            {'age_upon_outcome_in_weeks': {'$lte': 300}},
            {'$or': [
                {'breed': 'German Shepherd'},
                {'breed': 'Doberman Pinscher'},
                {'breed': 'Golden Retriever'},
                {'breed': 'Bloodhound'},
                {'breed': 'Rottweiler'}
            ]}
        ]}
    else:
        query = {}
        
    df = pd.DataFrame(shelter.read(query))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)
    
# Code for Datatable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [
        Input('datatable-id', 'selected_rows'),
        Input('datatable-id', 'selected_columns')
    ]
)
def update_styles(selected_rows, selected_columns):
    return [{       
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows] + [
    {       
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Code for Graphs
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')]
)
def update_graphs(view_data):
##FIX ME ####
# add code for chart of your choice (e.g. pie chart) #
    df = pd.DataFrame.from_dict(view_data)
    
    value_counts_series = df['breed'].value_counts(normalize=True).sort_index()
    figure = px.pie(
        df,
        names=value_counts_series.index,
        values=value_counts_series.array,
        title='Approx. Available Breeds'
    )
    return [
        dcc.Graph(            
            id='pie-chart',
            figure=figure
        )
    ]

# Code for Map (DONE)
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')]
)
def update_map(view_data):
    dff=pd.DataFrame.from_dict(view_data)
    
    markers = [dl.Marker(
        position=(row['location_lat'], row['location_long']),
        children=[
            dl.Tooltip(row['breed']),
            dl.Popup([
                html.H4('Animal Name'),
                html.P(row['name'])
            ])
        ]
    ) for (i, row) in dff.iterrows()]
    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[dl.TileLayer(id='base-layer-id')] + markers
        ) 
    ]

app